In [285]:
import folium
import pickle
import simplejson
import pandas as pd

# Make the grid

In [240]:
f=open('centers_long_lat_id_mesh_space=0.005.pickle','rb')
grid = pickle.load(f)
f.close()

In [270]:
grid_json = {"type":"FeatureCollection", "features":[]}

In [272]:
add_mesh = 0.005/2
for index, row in grid.iterrows():
    #print "index", index
    #print "row",row[0], row[1], row[2]
    coord = [[row[0]+add_mesh,row[1]+add_mesh],[row[0]+add_mesh,row[1]-add_mesh],[row[0]-add_mesh,row[1]-add_mesh],[row[0]-add_mesh,row[1]+add_mesh],[row[0]+add_mesh,row[1]+add_mesh]]
    dd = {"type":"Feature","id":str(int(row[2])),"properties":{"name":str(int(row[2]))},
         "geometry":{"type":"Polygon","coordinates":[coord]}
         }
    grid_json['features'].append(dd)
    #break

In [276]:
with open('grid.json', 'w') as outfile:
    simplejson.dump(grid_json, outfile)

# Make the dataframe

In [287]:
month = 2 #Feb
day_of_week = 1 #Tuesday
hour = 7 #From 7am to 8am

f=open('network_mesh_space=0.005_month='+str(month)+'_day='+str(day_of_week)+'.pickle','rb')
network = pickle.load(f)
f.close()

In [323]:
trips = {} #{center_id: [outgoing trips, incoming trips]}

count_pick_ups = network.loc[(hour,slice(None),slice(None),slice(None)),:]
count_pick_ups_index = count_pick_ups.index.values
for i in count_pick_ups_index:
    try:
        trips[str(int(i[2]))][0] += int(count_pick_ups.ix[i])
    except KeyError:
        trips[str(int(i[2]))] = [int(count_pick_ups.ix[i]),0]

count_drop_offs = network.loc[(slice(None),hour,slice(None),slice(None)),:]
count_drop_offs_index = count_drop_offs.index.values
for i in count_drop_offs_index:
    try:
        trips[str(int(i[3]))][1] += int(count_drop_offs.ix[i])
    except KeyError:
        trips[str(int(i[3]))] = [0,int(count_drop_offs.ix[i])]


In [332]:
tk = trips.keys()
tv = trips.values()

map_data = pd.DataFrame({'centers': tk, 'activity' : [i[0]+i[1] for i in tv], 
                         'attractiveness': [i[1]-i[0] for i in tv]})

In [333]:
map_data.head()

,activity,attractiveness,centers
0,3,3,9077
1,1,1,9074
2,17,-3,6513
3,1,1,4413
4,297,-59,6614


In [338]:
trips['6614']

[178, 119]

# Make the map

In [326]:
import folium

map_1 = folium.Map(location=[40.74006, -73.98605], zoom_start=14,
                   tiles='Stamen Terrain')
map_1.lat_lng_popover()

In [340]:
map_1.geo_json(geo_path='grid.json', data=map_data,
               columns=['centers', 'activity'],
               threshold_scale=[500, 1000, 1500, 2000, 2500, 3000],
               key_on='feature.id',fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
               legend_name='Activity Rate',reset=True)

#map.geo_json(geo_path=state_geo, data=state_data,
#             columns=['State', 'Unemployment'],
#             threshold_scale=[5, 6, 7, 8, 9, 10],
#             key_on='feature.id',
#             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
#             legend_name='Unemployment Rate (%)',
#             reset=True)


In [282]:
map_1.create_map(path='nyc.html')

In [330]:
map_1.geo_json?